In [1]:
from nsepy import get_history as gh
import datetime as dt
from matplotlib import pyplot as plt
from datetime import date
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pandas as pd
from numpy import asarray
import numpy as np
import tensorflow as tf
tf.random.set_seed(
   0
)
from yahoo_fin import stock_info
np.random.seed(0)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, save_model
#from tensorflow.keras.callbacks import ModelCheckpoint
import math
import yfinance as yf
from pandas_datareader import data as pdr
from keras.models import load_model
from datetime import datetime, timedelta


In [2]:
def pred(symbol):
    tf.random.set_seed(0)
    np.random.seed(0)
    start = dt.datetime(2016,1,1)
    today = date.today().strftime("%Y,%m,%d")
    year,mon,day=today.split(',')
    end = dt.datetime(int(year),int(mon),int(day))
   # end = dt.datetime(int(year),int(mon),int(day))- timedelta(days=2)
    print(end)
    stk_data =yf.download(tickers=symbol+'.NS',start=start,end=end, interval='1d')
    print(stk_data.tail())
   # stk_data = gh(symbol=symbol,start=start,end=end)
    #print(stk_data)
    sc = MinMaxScaler()
    Xt=np.array(stk_data['Open'])
    Yt=np.array(stk_data['Close'])
    X=[]
    for i in range(len(stk_data['Close'])):
        X.append(stk_data['Close'][i])
    for i in range(len(stk_data['Open'])):
        X.append(stk_data['Open'][i])

    sc.fit(np.array(X).reshape(-1, 1))
    xtrain, X_test, ytrain, y_test = train_test_split(Xt, Yt, random_state=0)
    X_test_scale= sc.transform(X_test.reshape(-1, 1))
    xtraining_set_scaled = sc.transform(np.array(xtrain).reshape(-1, 1))
    ytraining_set_scaled = sc.transform(ytrain.reshape(-1, 1))
    X_train = []
    y_train = []
    X_train= np.array(xtraining_set_scaled)
    y_train=np.array(ytraining_set_scaled)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    t=math.inf
    n_split=3

    for train_index,test_index in KFold(n_split,shuffle = False).split(X_train):
        xtrain,xtest=X_train[train_index],X_train[test_index]
        ytrain,ytest=y_train[train_index],y_train[test_index]
        keras.backend.clear_session()
        regressor = Sequential()
        
        regressor.add(LSTM(units = 256, return_sequences = True, input_shape = (xtrain.shape[1], 1)))
        regressor.add(Dropout(0.25))
        regressor.add(LSTM(units = 256,  return_sequences = True))
        regressor.add(Dropout(0.25))
        regressor.add(LSTM(units = 256,  return_sequences = True))
        regressor.add(Dropout(0.25))

        regressor.add(LSTM(units = 256))
        regressor.add(Dropout(0.25))
    
        regressor.add(Dense(units = 512))
        regressor.add(Dense(units = 128))
        regressor.add(Dense(units = 8))
        regressor.add(Dense(units = 1))

        regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

        regressor.fit(xtrain, ytrain, epochs = 25, batch_size = 32,verbose=0)
    
        scores=regressor.evaluate(xtest,ytest)
        if scores<t:
            t=scores
            filepath = symbol
            save_model(regressor, filepath, save_format='h5')
    keras.backend.clear_session()
    regressor = load_model(symbol)
    #year,mon,day=today.split(',')
    #today = date.today().strftime("%Y,%m,%d")
    #st = dt.datetime(int(year),int(mon),int(day))- timedelta(days=1)
    #end = dt.datetime(int(year),int(mon),int(day))
    #print("End",end)
    #yf.pdr_override()
    #totest=pdr.get_data_yahoo(symbol+".NS", start=st, end=end)
   # print(totest)
    #cl=totest['Open']
    #cl=[490.95,634.96,8465,479.54,608]
    data = yf.download(tickers=symbol+'.NS', period='1d', interval='15m')
    predicted_stock_price = regressor.predict(sc.transform(np.array(data['Open'][1]).reshape(-1, 1)))
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    print(data['Open'][1],symbol,"price is",predicted_stock_price,"for the date",end)
    

In [3]:
#stocks=['ICICISENSX','SBIN','TATAELXSI','MARICO','WIPRO','ADANIPORTS','HDFCLIFE','ICICI','AXISBANK','LAURUSLABS',
 #      'IRCTC','IEX','SUNPHARMA','LXCHEM','ZENSARTECH','JSWENERGY','MFSL','HINDALCO','IDFC','TRIDENT','TATAPOWER',
  #     'HINDZINC','IRFC','CREDITACC','RAILTEL','SJS','NAM-INDIA']
stocks=['ICICISENSX','SBIN','TATAELXSI','MARICO','WIPRO']
for i in stocks:
    pred(i)

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low       Close   Adj Close  Volume
Date                                                                       
2022-03-31  643.98999  643.98999  635.00000  636.119995  636.119995    2159
2022-04-01  640.98999  645.00000  615.00000  641.679993  641.679993    2578
2022-04-04  636.01001  662.98999  636.01001  658.039978  658.039978    8625
2022-04-05  658.50000  661.00000  653.01001  655.349976  655.349976    9759
2022-04-06  659.97998  659.97998  648.00000  650.289978  650.289978    2036
5/5 [==============================] - 2s 11ms/step - loss: 3.9351e-04
[*********************100%***********************]  1 of 1 completed
647.0 ICICISENSX price is [[647.49365]] for the date 2022-04-07 00:00:00
2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Dat

In [4]:
stocks=['ADANIPORTS','HDFCLIFE','AXISBANK','LAURUSLABS']
for i in stocks:
    pred(i)
    

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-03-31  767.400024  781.000000  765.500000  774.200012  774.200012   
2022-04-01  774.500000  790.700012  774.500000  785.750000  785.750000   
2022-04-04  791.000000  822.150024  783.650024  818.549988  818.549988   
2022-04-05  840.000000  860.900024  826.000000  848.049988  848.049988   
2022-04-06  852.950012  862.650024  840.000000  849.849976  849.849976   

              Volume  
Date                  
2022-03-31   5468951  
2022-04-01   6816743  
2022-04-04  13743062  
2022-04-05  21090646  
2022-04-06   8785541  
13/13 [==============================] - 2s 13ms/step - loss: 2.0346e-04
[*********************100%***********************]  1 of 1 completed
857.0499877929688 ADANIPORTS price is [[861.2065]] for the date 2022-04-07 00:00:00
2022-

In [5]:
stocks=['HINDZINC','IRFC','CREDITACC','RAILTEL','SJS','NAM-INDIA']
for i in stocks:
    pred(i)

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-03-31  312.500000  314.299988  308.200012  310.149994  310.149994   
2022-04-01  311.549988  321.549988  309.950012  320.649994  320.649994   
2022-04-04  323.500000  326.799988  321.250000  325.049988  325.049988   
2022-04-05  330.000000  336.000000  325.049988  332.850006  332.850006   
2022-04-06  332.000000  335.000000  327.000000  333.399994  333.399994   

             Volume  
Date                 
2022-03-31  1148619  
2022-04-01  1339271  
2022-04-04   797433  
2022-04-05  2069155  
2022-04-06   732422  
13/13 [==============================] - 2s 8ms/step - loss: 7.2746e-04
[*********************100%***********************]  1 of 1 completed
333.6499938964844 HINDZINC price is [[334.9427]] for the date 2022-04-07 00:00:00
2022-04-07 00:0

In [6]:
stocks=['IRCTC','IEX','SUNPHARMA','LXCHEM','ZENSARTECH','JSWENERGY']
for i in stocks:
    pred(i)

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-03-31  791.049988  792.750000  773.000000  774.700012  774.700012   
2022-04-01  777.000000  802.650024  775.549988  798.200012  798.200012   
2022-04-04  803.950012  837.700012  803.000000  832.700012  832.700012   
2022-04-05  840.849976  840.950012  819.450012  822.349976  822.349976   
2022-04-06  820.000000  828.950012  787.750000  791.950012  791.950012   

             Volume  
Date                 
2022-03-31  2496423  
2022-04-01  4160527  
2022-04-04  5650502  
2022-04-05  3480544  
2022-04-06  5400080  
5/5 [==============================] - 2s 11ms/step - loss: 1.2715e-04
[*********************100%***********************]  1 of 1 completed
791.8499755859375 IRCTC price is [[788.2223]] for the date 2022-04-07 00:00:00
2022-04-07 00:00:00

In [7]:
stocks=['MFSL','HINDALCO','IDFC','TRIDENT','TATAPOWER','SBICARD']
for i in stocks:
    pred(i)

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2022-03-31  747.450012  765.000000  743.799988  753.900024  753.900024  843788
2022-04-01  751.500000  777.000000  751.500000  774.250000  774.250000  520790
2022-04-04  773.799988  795.750000  773.799988  791.250000  791.250000  345275
2022-04-05  791.250000  794.900024  772.349976  779.849976  779.849976  936951
2022-04-06  781.000000  791.000000  769.000000  776.599976  776.599976  336563
13/13 [==============================] - 2s 10ms/step - loss: 6.5329e-04
[*********************100%***********************]  1 of 1 completed
774.0999755859375 MFSL price is [[797.0996]] for the date 2022-04-07 00:00:00
2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low   

In [8]:
stocks=['HCLTECH','INDUSINDBK','TATASTEEL','CIPLA','ESCORTS','JKCEMENT','KOTAKBANK']
for i in stocks:
    pred(i)

2022-04-07 00:00:00
[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2022-03-31  1174.800049  1174.800049  1161.000000  1163.750000  1163.750000   
2022-04-01  1161.000000  1174.000000  1152.050049  1169.650024  1169.650024   
2022-04-04  1173.199951  1188.949951  1168.000000  1187.050049  1187.050049   
2022-04-05  1190.000000  1203.000000  1184.099976  1195.449951  1195.449951   
2022-04-06  1190.000000  1194.900024  1167.500000  1170.500000  1170.500000   

             Volume  
Date                 
2022-03-31  3006639  
2022-04-01  2479745  
2022-04-04  2282137  
2022-04-05  2634464  
2022-04-06  2838968  
13/13 [==============================] - 1s 11ms/step - loss: 2.8412e-04
[*********************100%***********************]  1 of 1 completed
1169.0 HCLTECH price is [[1157.4961]] for the date 2022-04-07

In [2]:
def notrain(symbol):
    keras.backend.clear_session()
    tf.random.set_seed(0)
    np.random.seed(0)
    regressor = load_model(symbol)
    start = dt.datetime(2016,1,1)
    today = date.today().strftime("%Y,%m,%d")
    year,mon,day=today.split(',')

    end = dt.datetime(int(year),int(mon),int(day))
    #print(end)
    #stk_data = gh(symbol=symbol,start=start,end=end)
    stk_data = yf.download(tickers=symbol+'.NS', period='6y', interval='1d')

    sc = MinMaxScaler()
    Xt=np.array(stk_data['Open'])
    Yt=np.array(stk_data['Close'])
    X=[]
    for i in range(len(stk_data['Close'])):
        X.append(stk_data['Close'][i])
    for i in range(len(stk_data['Open'])):
        X.append(stk_data['Open'][i])

    sc.fit(np.array(X).reshape(-1, 1))
    
    
    today = date.today().strftime("%Y,%m,%d")
    year,mon,day=today.split(',')
    st = dt.datetime(int(year),int(mon),int(day))- timedelta(days=2)
    end = dt.datetime(int(year),int(mon),int(day))
    #stk_data2 = gh(symbol=symbol,start=st,end=end)
    stk_data2 = yf.download(tickers=symbol+'.NS', period='3d', interval='1d')

    X_train=[]
    y_train=[]
    X_train=np.array(stk_data2['Open'])
    y_train=np.array(stk_data2['Close'])
    #print(X_train,y_train)
    xtraining_set_scaled = sc.transform(np.array(X_train).reshape(-1, 1))
    ytraining_set_scaled = sc.transform(y_train.reshape(-1, 1))
    X_train= np.array(xtraining_set_scaled)
    y_train=np.array(ytraining_set_scaled)
    #print(X_train,y_train)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    #print(st,stk_data2)
   # regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    #regressor.fit(X_train, y_train, epochs = 25, batch_size = 32,verbose=0)
    filepath = symbol+"2"
    #save_model(regressor, filepath, save_format='h5')
    #regressor = load_model(symbol+"2")

    ##totest=pdr.get_data_yahoo(symbol+".NS", start=st, end=end)
    #cl=totest['Open']
    #cl=[490.95,634.96,8465,479.54,608]
    data = yf.download(tickers=symbol+'.NS', period='1d', interval='15m')
    #print(data)
    #print("test data",sc.transform(np.array(data['Open'][0]).reshape(-1, 1)))

    predicted_stock_price = regressor.predict(sc.transform(np.array(data['Open'][1]).reshape(-1, 1)))
    #print(predicted_stock_price)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    print(data['Open'][1],symbol,"price is",predicted_stock_price,"for the date",end)


In [3]:

stocks=['ICICISENSX','SBIN','TATAELXSI','MARICO','WIPRO','ADANIPORTS','HDFCLIFE','AXISBANK','LAURUSLABS',
       'IRCTC','SUNPHARMA','ZENSARTECH','MFSL','HINDALCO','IDFC','TATAPOWER',
       'HINDZINC','IRFC','CREDITACC','RAILTEL','SJS','NAM-INDIA','HCLTECH','INDUSINDBK','TATASTEEL','JKCEMENT','KOTAKBANK'
        ,'SBICARD','ESCORTS','LXCHEM','CIPLA']
#stocks=['ICICISENSX','SBIN','TATAELXSI','MARICO','WIPRO']
for i in stocks:
    notrain(i)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
644.25 ICICISENSX price is [[644.78973]] for the date 2022-04-08 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
515.7999877929688 SBIN price is [[523.107]] for the date 2022-04-08 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
8841.099609375 TATAELXSI price is [[8435.487]] for the date 2022-04-08 00:00:00
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********

KeyboardInterrupt: 

In [11]:
   ''' today = date.today().strftime("%Y,%m,%d")
    year,mon,day=today.split(',')
    
    st = dt.datetime(int(year),int(mon),int(day)-3)
    end = dt.datetime(int(year),int(mon),int(day)+1)
    #print("End",end)
    yf.pdr_override()
    totest=pdr.get_data_yahoo("ICICISENSX"+".NS", start=st, end=end)
    print(totest)
    '''

' today = date.today().strftime("%Y,%m,%d")\n year,mon,day=today.split(\',\')\n \n st = dt.datetime(int(year),int(mon),int(day)-3)\n end = dt.datetime(int(year),int(mon),int(day)+1)\n #print("End",end)\n yf.pdr_override()\n totest=pdr.get_data_yahoo("ICICISENSX"+".NS", start=st, end=end)\n print(totest)\n '

In [12]:
start = dt.datetime(2016,1,1)
today = date.today().strftime("%Y,%m,%d")
year,mon,day=today.split(',')

end = dt.datetime(int(year),int(mon),int(day))
stk_data = gh(symbol='ADANIPORTS',start=start,end=end)
    
sc = MinMaxScaler()
Xt=np.array(stk_data['Open'])
Yt=np.array(stk_data['Close'])
X=[]
for i in range(len(stk_data['Close'])):
    X.append(stk_data['Close'][i])
for i in range(len(stk_data['Open'])):
    X.append(stk_data['Open'][i])

sc.fit(np.array(X).reshape(-1, 1))
data = yf.download(tickers='ADANIPORTS.NS', period='2d', interval='1ds')
print(data)
print(data['Open'][0])
print(stock_info.get_live_price("ADANIPORTS.NS"))
regressor = load_model('ADANIPORTS')
predicted_stock_price = regressor.predict(sc.transform(np.array(783).reshape(-1, 1)))
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ADANIPORTS.NS: Invalid input - interval=1ds is not supported. Valid intervals: , 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
  '''today = date.today().strftime("%Y,%m,%d")

year,mon,day=today.split(',')
st = dt.datetime(int(year),int(mon),int(day)-3)
end = dt.datetime(int(year),int(mon),int(day)+1)
totest=pdr.get_data_yahoo("ICICISENSX"+".NS", start=st, end=end)
print(totest)
cl=totest['Open']
  '''

In [ ]:
start = dt.datetime(2016,1,1)
today = date.today().strftime("%Y,%m,%d")
year,mon,day=today.split(',')

end = dt.datetime(int(year),int(mon),int(day))
data = yf.download(tickers='ADANIPORTS.NS', start=start,end=end, interval='90m')
print(data)

In [ ]:
start = dt.datetime(2016,1,1)
today = date.today().strftime("%Y,%m,%d")
year,mon,day=today.split(',')
end = dt.datetime(int(year),int(mon),int(day))
data = yf.download(tickers='ICICISENSX'+'.NS',start=start,end=end, interval='1d')
print(data.tail())


In [ ]:
print(stock_info.get_live_price("ADANIPORTS.NS"))

In [ ]:
start = dt.datetime(2016,1,1)
today = date.today().strftime("%Y,%m,%d")
year,mon,day=today.split(',')
end = dt.datetime(int(year),int(mon),int(day))
   # end = dt.datetime(int(year),int(mon),int(day))- timedelta(days=2)
print(end)
stk_data =yf.download(tickers=+'.NS',start=start,end=end, interval='1d')
print(stk_data)